# FLow with Multiple Stocks - Duality

In this nb, we solve the cutting optima problem with big order quantity to cut many specs at the same time with many stocks ( stock with same width and nearly same weight)
The condition/constraints as below: 
- `Generate Pattern` that fit in stock choice and sum each FG cut > lower bound demand
- `Filter Pattern` that keep the trim_cost <= 4%
- `Cut Pattern` that cut many stocks with filtered_pattern to minimize the stocks used

## >> load configs and params

In [1]:
## PARAMETER 
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH590R-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 2
          ,"maker" : "CSC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2          ->>>> OPERATOR FLOW
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

import json

with open('margin_dict.json', 'r') as file:
    margin_dict = json.load(file)

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:6
BOUND_VALUE:None


#### Assume that we have already process data and have a job list, AND THE OPERATOR assigns the bound for cut else where, now we load each job and proceed optimization

In [2]:
stocks = {'TP235H002656-2': {'receiving_date': 45150, 'width': 1219, 'weight': 4332.5},
 'TP235H002656-1': {'receiving_date': 44951, 'width': 1219, 'weight': 4332.5},
 'TP232H001074': {'receiving_date': 45040, 'width': 1233, 'weight': 7105.0},
 'TP232H001073': {'receiving_date': 45153, 'width': 1233, 'weight': 7550.0},
 'TP236H005198': {'receiving_date': 45011, 'width': 1136, 'weight': 8000.0},
 'TP235H002652': {'receiving_date': 45039, 'width': 1219, 'weight': 8400.0},
 'TP235H002654': {'receiving_date': 45045, 'width': 1219, 'weight': 8500.0},
 'TP232H001072': {'receiving_date': 45013, 'width': 1233, 'weight': 8675.0},
 'TP235H002655': {'receiving_date': 45229, 'width': 1219, 'weight': 8845.0},
 'TP235H002653': {'receiving_date': 45045, 'width': 1219, 'weight': 8855.0},
 'TP232H001075': {'receiving_date': 45247, 'width': 1233, 'weight': 9630.0}
 }

finish = {'F23': {'width': 306.0,  'need_cut': 839.0,  'upper_bound': 1548.5841833599998,
  'fc1': 2365.2806112,  'fc2': 3692.5657704,  'fc3': 3457.8613836},
 'F22': {'width': 205.0,  'need_cut': 498.7908121410992,  'upper_bound': 3362.2258921410994,
  'fc1': 9544.7836,  'fc2': 5494.6232,  'fc3': 3908.5464},
 'F21': {'width': 188.0,  'need_cut': 30772.599709771595,  'upper_bound': 39966.4243228516,
  'fc1': 30646.0820436,  'fc2': 35762.3146452,  'fc3': 34039.2591132},
 'F20': {'width': 175.0,  'need_cut': 28574.78588807786,  'upper_bound': 36618.447115077855,
  'fc1': 26812.20409,  'fc2': 31288.38713,  'fc3': 29780.88883},
 'F19': {'width': 155.0,  'need_cut': 4401.8405357987585,  'upper_bound': 5851.570175548759,
  'fc1': 4832.4321325,  'fc2': 5639.1860525,  'fc3': 5367.4857775},
 'F18': {'width': 133.0,  'need_cut': 400.0,  'upper_bound': 795.8254562499999,
  'fc1': 1319.4181875,  'fc2': 759.546375,  'fc3': 540.295875},
 'F17': {'width': 120.0,  'need_cut': 1751.0,  'upper_bound': 2526.6533504,
  'fc1': 2585.511168,  'fc2': 4319.793456,  'fc3': 3797.778504},
 'F24': {'width': 82.0,  'need_cut': 977.9362646180011,  'upper_bound': 1585.531389098001,
  'fc1': 2025.3170816,  'fc2': 3383.8382072,  'fc3': 2974.9264948}
  }

#### >> GENERATE PATTERN BY DUALITY

In [3]:
from typing import Dict, Any
import copy

from auxilaries import *
from solvers import *

# Setup GLOBAL problem varibles - KEEP OUT OF THE LOOP
print("PATTERN GENERATION DUALITY", end=".")
final_solution_patterns =[]

# CAN BE RESET AT EACH ROUND
total_dual_pat = []
dual_stocks = copy.deepcopy(stocks)
dual_finish = copy.deepcopy(finish)
nai_patterns = make_naive_patterns(stocks, finish, MIN_MARGIN)

PATTERN GENERATION DUALITY.

In [24]:
# RESET AT EACH ROUND
total_dual_pat = []
del max_key, dual_finish,dual_stocks, nai_patterns
dual_stocks = copy.deepcopy(stocks_cont)
dual_finish = copy.deepcopy(finish_cont)
nai_patterns = make_naive_patterns(dual_stocks, dual_finish, MIN_MARGIN)

In [25]:
# CUTTING-STOCK-LOOP
i = 0
rm_stock = True
while rm_stock == True:
    # print("PHASE 0: setup stock")

    try:
        # remove stock that has many pattern over other stocks
        dual_stocks = filter_out_stock_by_cond(dual_stocks,max_key)
        # print(f"remove stock key {max_key}")
    except NameError:
        print('->> max_key error, next')
        pass

    len_stocks = len(dual_stocks)

    # print("PHASE 1: Naive/ Dual Pattern Generation",end=".")    
    patterns = make_naive_patterns(dual_stocks, dual_finish, MIN_MARGIN)
    dual_finish = create_finish_demand_by_line_fr_naive_pattern(patterns, dual_finish)

    # print("PHASE 2: Pattern Duality",end=".")
    new_pattern = generate_pattern_dual(dual_stocks, dual_finish, patterns, MIN_MARGIN) # Stock nao do toi uu hon stock khac o width thi new pattern luon bi chon cho stock do #FIX
    dual_pat = []
    while new_pattern not in dual_pat:
        patterns.append(new_pattern)        # pattern de generate them new pattern
        total_dual_pat.append(new_pattern)  # tinh tong dual pattern co the duoc generate
        dual_pat.append(new_pattern)        # dual pat de tinh stock bi lap nhieu lan
        new_pattern = generate_pattern_dual(dual_stocks, dual_finish, patterns, MIN_MARGIN)
        print(end=".")

    # filter stock having too many patterns
    ls = count_pattern(dual_pat)
    max_key = max(ls, key=ls.get) # take the max 
    max_count = ls[max_key]
    if max_count > 1 and i < len_stocks - 2:# con lai it nhat 2 stock
        rm_stock = True
        i +=1
        print(f"{i} round")
    else: 
        rm_stock = False

# Phase 1.2: Combine pattterns
sum_patterns = nai_patterns + total_dual_pat

print()
print(f'>> Total naive/dual patterns: {len(sum_patterns)}')

# Phrase: Filter Patterns having trim loss as requirements
print("PHASE 3: Filter Patterns", end=":")
filtered_trimloss_pattern = []
idx=0
for pattern in sum_patterns:
    cuts_dict= pattern['cuts']
    width_s = stocks[pattern['stock']]['width']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in cuts_dict.keys()])
    trim_loss_pct = round(trim_loss/width_s * 100, 3)
    if trim_loss_pct <= 4.000: # filter for naive pattern
        idx +=1
        pattern.update({'trim_loss':trim_loss, "trim_loss_pct": trim_loss_pct, "patt_id":idx})
        filtered_trimloss_pattern.append(pattern)
        print(end=".")
print()
print(f">> The Patterns with less 4% trim loss: {len(filtered_trimloss_pattern)}")

->> max_key error, next
..
>> Total naive/dual patterns: 5
PHASE 3: Filter Patterns:..
>> The Patterns with less 4% trim loss: 2


In [26]:
# Phrase 4: Cut Pattern to map all pattern to demand requirements
print("PHASE 4: Cut Patterns", end=".")
print()
filtered_stocks = [filtered_trimloss_pattern[i]['stock'] for i in range(len(filtered_trimloss_pattern))]
chosen_stocks = {}
for stock_name, stock_info in stocks.items():
    if stock_name in filtered_stocks:
        chosen_stocks[stock_name]= {**stock_info}

# CHU Y: co truong hop CHOSEN stock < total stock need to be cut
print(f">> Stocks with fit patterns: {chosen_stocks.keys()}") # 

solution, total_cost = cut_weight_patterns(chosen_stocks, dual_finish, filtered_trimloss_pattern)
solution_list = []
print(f"Total stock need: {total_cost}")
for i, pattern_info in enumerate(filtered_trimloss_pattern):
    count = solution[i]
    # rcount = 1 if 0 < count <= 1.0 else round(count)
    if count > 0:
        # cut_pattern = pattern_info['cuts']
        s = pattern_info['stock']
        # print(f"+ Take ({rcound}), {pattern_info}")
        sol = {"count": count, **pattern_info}
        solution_list.append(sol)

PHASE 4: Cut Patterns.
>> Stocks with fit patterns: dict_keys(['TP235H002653'])
Total stock need: 2


In [27]:
# Neu lap stock thi rm all pattern tru cai trim loss thap nhat va chay lai
sorted_solution_list = sorted(solution_list, key=lambda x: (x['stock'],  x.get('trim_loss_pct', float('inf'))))

# now take first overused stock pattern only.
overused_list = []
take_stock = None
for solution_pattern in sorted_solution_list:
    current_stock = solution_pattern['stock']
    if current_stock == take_stock:
        overused_list.append(solution_pattern)
    else:
        take_stock = current_stock
        final_solution_patterns.append(solution_pattern)
            
print(f'Take {len(final_solution_patterns)} possible stocks-patterns')

# Phase 5: Evaluation Over-cut / Stock Ratio
print("PHASE 5: Evaluation Stock Ratio", end=".")
for i, sol in enumerate(final_solution_patterns):
    s = final_solution_patterns[i]['stock'] # stock cut
    cuts_dict = final_solution_patterns[i]['cuts']
    weight_dict = {f: round(cuts_dict[f] * finish[f]['width']*stocks[s]['weight']/stocks[s]['width'],3) for f in cuts_dict.keys()}
    # print(weight_dict)
    final_solution_patterns[i] = {**sol,
                        "cut_w": weight_dict}
# Total Cuts
total_sums = count_weight(final_solution_patterns)
print(f">> Total cut of each Finished Goods: {total_sums}")

over_cut = {}
over_cut_ratio = {}
# Overcut cutweight (slice * fg_width * wu) - need_cut 
for key in total_sums.keys():
    over_cut[key] = round(total_sums[key] - finish[key]['need_cut'],3)
    over_cut_ratio[key] = round(over_cut[key]/finish[key]['fc1'], 4)

print(f">> Over-cut: {over_cut}")
print(f">> Stock-ratio on next month forecast: {over_cut_ratio}")

Take 10 possible stocks-patterns
PHASE 5: Evaluation Stock Ratio.>> Total cut of each Finished Goods: {'F23': 2108.61, 'F22': 1442.32, 'F21': 29626.02, 'F20': 38275.17999999999, 'F19': 4926.2300000000005, 'F18': 1932.26, 'F17': 2132.49, 'F24': 2396.28}
>> Over-cut: {'F23': 1269.61, 'F22': 943.529, 'F21': -1146.58, 'F20': 9700.394, 'F19': 524.389, 'F18': 1532.26, 'F17': 381.49, 'F24': 1418.344}
>> Stock-ratio on next month forecast: {'F23': 0.5368, 'F22': 0.0989, 'F21': -0.0374, 'F20': 0.3618, 'F19': 0.1085, 'F18': 1.1613, 'F17': 0.1475, 'F24': 0.7003}


In [28]:
# CHECK FOR ANOTHER ROUND
if not overused_list:
    print("FINISH CUTTING")
else:
    # go back to PHRASE 3
    print("BACK TO PHRASE 3: TO FILTER OUT PATTERNS")
    taken_stocks = [p['stock'] for p in final_solution_patterns]
    finish_cont = {}
    for f, f_info in dual_finish.items():
        if over_cut[f] < 0:
            f_info['need_cut'] = over_cut[f] *(-1)
            finish_cont[f] ={**f_info}

    stocks_cont = {}
    for s, s_info in dual_stocks.items():
        if s not in taken_stocks:
            stocks_cont[s] = {**s_info}

FINISH CUTTING


In [8]:
[p['stock'] for p in final_solution_patterns] # ROUND 1

['TP232H001072',
 'TP232H001073',
 'TP232H001074',
 'TP235H002656-1',
 'TP235H002656-2',
 'TP236H005198']

In [14]:
[p['stock'] for p in final_solution_patterns] # ROUND 2

['TP232H001072',
 'TP232H001073',
 'TP232H001074',
 'TP235H002656-1',
 'TP235H002656-2',
 'TP236H005198',
 'TP235H002652',
 'TP235H002654']

In [22]:
[p['stock'] for p in final_solution_patterns] # ROUND 3

['TP232H001072',
 'TP232H001073',
 'TP232H001074',
 'TP235H002656-1',
 'TP235H002656-2',
 'TP236H005198',
 'TP235H002652',
 'TP235H002654',
 'TP235H002655']

In [29]:
[p['stock'] for p in final_solution_patterns] # ROUND 4

['TP232H001072',
 'TP232H001073',
 'TP232H001074',
 'TP235H002656-1',
 'TP235H002656-2',
 'TP236H005198',
 'TP235H002652',
 'TP235H002654',
 'TP235H002655',
 'TP235H002653']

['TP232H001072', 'TP232H001073', 'TP232H001074', 'TP235H002656-1', 'TP235H002656-2', 'TP236H005198', 'TP232H001075', 'TP235H002652', 'TP235H002654']